This is a notebook that can be used to pull data from the iRacing API.

https://forums.iracing.com/discussion/15068/general-availability-of-data-api/p1

uses the [iracingdataapi package](https://github.com/jasondilworth56/iracingdataapi) to pull data from the iRacing API.



In [ ]:
!pip install git+https://github.com/jasondilworth56/iracingdataapi.git


In [43]:
import csv
import json
from iracingdataapi.client import irDataClient

import os

Race = 5

username = os.getenv("IRACING_USERNAME")
password = os.getenv("IRACING_PASSWORD")


# alternatively you can set the username and password directly here
# fill in your username and password, an uncomment the lines below
# username = "putyourusernamehereitisusuallyyouremailaddress"
# password = "putyourpasswordhere"

def calculate_f499_points(start, finish, incident_count):
    return (start - finish) + (4 if incident_count == 0 else -1 * (2 * incident_count))


def get_session_link(subsession_id, new_ui=False):
    if new_ui:
        format = "https://members-ng.iracing.com/racing/results-stats/results?subsessionid="
    else:
        format = "https://members.iracing.com/membersite/member/EventResult.do?&subsessionid="

    return f"{format}{subsession_id}"


def last_10_results(client: irDataClient, customer_id=None):
    # if you do not pass a customer_id as the param to the method call, it will default to the customer_id of the user you are logged in as
    # note: this API call returns only the last 10 races, I think.
    stats = client.stats_member_recent_races(customer_id)
    # get the stats for the last 10 races
    last_10_races = stats['races']
    # iterate over the last 10 races
    for i, race in enumerate(last_10_races, start=1):
        start_time = race['session_start_time']
        subsession_id = race['subsession_id']
        start_position = race['start_position']
        finish_position = race['finish_position']
        incident_count = race['incidents']
        series_name = race['series_name']
        track_name = race['track']['track_name']
        points = calculate_f499_points(start_position, finish_position, incident_count)

        print_race_results(subsession_id, start_time, start_position, finish_position, incident_count, points,
                           series_name, track_name)
    print(f"Total Races: {len(last_10_races)}")


def write_results_to_file(results, filename):
    # Write the raw results to a JSON file
    with open(f"{filename}_raw_results.json", 'w') as jsonfile:
        json.dump(results, jsonfile)


def write_results_to_csv_file(race_data_list, filename):
    # Specify the order of your columns
    fieldnames = ["season_year",
                  "season_quarter",
                  "week_number",
                  "series_name",
                  "series_id",
                  "start_time",
                  "track_name",
                  "session_link",
                  "subsession_id",
                  "start_position",
                  "finish_position",
                  "incident_count", 
                  "_499_points"
                  ]

    # Open your CSV file in write mode
    with open(f'{filename}_data.csv', 'w', newline='') as csvfile:
        # Create a CSV writer object
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        # Write the header to the CSV file
        writer.writeheader()

        # Write each dictionary in the list to the CSV file
        for race_data_dict in race_data_list:
            writer.writerow(race_data_dict)


def print_race_results(race_data):
    print(f"Race {race_data['start_time']}:")
    print(f"Session Link: [{race_data['subsession_id']}]({race_data['session_link']})")
    print(f"Series: {race_data['series_name']}")
    print(f"Track: {race_data['track_name']}")
    print(f"Start Position: {race_data['start_position'] if race_data['start_position'] else 'N/A'}")
    print(f"Finish Position: {race_data['finish_position'] if race_data['finish_position'] else 'N/A'}")
    print(f"Number of Incidents: {race_data['incident_count']}")
    print(f"F499 Scoring Points: {race_data['_499_points']}")
    print()


def get_499_series(client: irDataClient):
    series = client.series
    # print out the series
    # find the series with the string "LMP3" in the name
    _499_series = [s for s in series if "LMP3" in s['series_name']]
    # # add any series that has the string "LMP2" in the name to the list
    _499_series.extend([s for s in series if "LMP2" in s['series_name']])
    # # add any series that has the string "F4" in the name to the list
    _499_series.extend([s for s in series if "FIA Formula 4" in s['series_name']])
    # # add any series that has the string "Falken Sports Car" in the name to the list
    _499_series.extend([s for s in series if "Falken Tyre Sports Car" in s['series_name']])
    # add any series that has the string "IMSA" in the name to the list
    _499_series.extend([s for s in series if "IMSA iRacing Series" in s['series_name']])

    # put the series_id and series_name in a tuple and add them to a list
    _499_series = [(s['series_id'], s['series_name']) for s in _499_series]
    return _499_series


def construct_499_race_data(raw_result):
    start_position = raw_result['starting_position_in_class'] + 1
    finish_position = raw_result['finish_position_in_class'] + 1
    incident_count = raw_result['incidents']
    track_name = raw_result['track']['track_name']
    subsession_id = raw_result['subsession_id']
    start_time = raw_result['start_time']
    week_number = raw_result['race_week_num'] + 1
    season_year = raw_result['season_year']
    season_quarter = raw_result['season_quarter']
    _499_points = calculate_f499_points(start_position, finish_position, incident_count)
    session_link = get_session_link(subsession_id, True)

    return {
        "start_position": start_position,
        "finish_position": finish_position,
        "incident_count": incident_count,
        "track_name": track_name,
        "subsession_id": subsession_id,
        "start_time": start_time,
        "week_number": week_number,
        "season_year": season_year,
        "season_quarter": season_quarter,
        "_499_points": _499_points,
        "session_link": session_link,
        "series_name": raw_result['series_name'],
        "series_id": raw_result['series_id']
    }


def run():
    cust_id = 929408  # Hannah
    cust_id = 643506  # Tom
    cust_id = 818734  # JD
    
    desired_season_year = 2024
    desired_season_quarter = 3
    desired_season_week = 1

    idc = irDataClient(username=username, password=password)
    # last_10_results(idc, cust_id)
    series_of_interest = get_499_series(idc)

    raw_results = []
    race_data_list = []

    title = f"{desired_season_year}S{desired_season_quarter} Week {desired_season_week} CustID {cust_id}"
    print(f"Customer ID: {cust_id}, {title}")

    for series_id, series_name in series_of_interest:
        # NOTE: the week number is 0 based, so week 1 is actually week 0.
        # I handle that here by subtracting 1 from the desired week number
        series_results = idc.result_search_series(cust_id=cust_id,
                                                  series_id=series_id,
                                                  official_only=True,
                                                  event_types=[Race],
                                                  season_year=desired_season_year,
                                                  season_quarter=desired_season_quarter,
                                                  race_week_num=desired_season_week - 1)
        print(f"Series: {series_name}")
        print(f"Series ID: {series_id}")
        print(f"{len(series_results)} Races")
        print("=" * 36)
        # print the series results
        for i, result in enumerate(series_results, start=1):
            raw_results.append(result)
            race_data = construct_499_race_data(result)
            race_data_list.append(race_data)
            
            print_race_results(race_data)
            print("-" * 36)

        print("=" * 36)
        print()
        
        # sort race_data_list by start_time, descending
        race_data_list.sort(key=lambda x: x['start_time'])

        write_results_to_csv_file(race_data_list, title.replace(" ", "_"))
        write_results_to_file(raw_results, title.replace(" ", "_"))


run()


Customer ID: 818734, 2024S3 Week 1 CustID 818734
Series: LMP3 Trophy - Fixed
Series ID: 525
2 Races
Race 2024-06-13T18:30:00Z:
Session Link: [69424553](https://members-ng.iracing.com/racing/results-stats/results?subsessionid=69424553)
Series: LMP3 Trophy - Fixed
Track: Motorsport Arena Oschersleben
Start Position: 10
Finish Position: 13
Number of Incidents: 0
F499 Scoring Points: 1

------------------------------------
Race 2024-06-18T00:30:00Z:
Session Link: [69518923](https://members-ng.iracing.com/racing/results-stats/results?subsessionid=69518923)
Series: LMP3 Trophy - Fixed
Track: Watkins Glen International
Start Position: 16
Finish Position: 14
Number of Incidents: 4
F499 Scoring Points: -6

------------------------------------

Series: LMP2 Prototype  Challenge – Fixed
Series ID: 457
2 Races
Race 2024-06-13T22:00:00Z:
Session Link: [69428008](https://members-ng.iracing.com/racing/results-stats/results?subsessionid=69428008)
Series: LMP2 Prototype  Challenge – Fixed
Track: Watkin